# Strcuture builder for gernerating the endmember, dilute and SQS structure

## 1. Import necessay packages

In [33]:
from prl_structure import PRLStructure
from structure_builder.structure_tools import sort_x_by_y, canonicalize_config, get_density_from_pt, get_ele_list_from_struct,scale_struct,gen_replacement_dict,substitute_configuration,substitute_configuration_with_metadata
from endmember_tools import get_sublattice_information, get_templates
from endmember_tools import get_endmembers_with_templates
from dilute_structure_tools import dilute_substitution
from database_tools import get_structures_from_database
from sqs_tools import AbstractSQS, enumerate_sqs
from pymatgen.core import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from collections import Counter
from pymatgen.core.periodic_table import Element
from itertools import permutations, product, chain
import copy
import itertools
import pymatgen as pmg


## 2. Endmembers generation 

### 2.1 Load the POSCAR template of target phase

In [24]:
POSCAR_name = 'Al3Ni2.poscar'
str_name = 'Al3Ni2'

with open(POSCAR_name, 'r', encoding='utf-8') as file:
    POSCAR_STR = file.read()
structure = Structure.from_str(POSCAR_STR, fmt='POSCAR')

In [25]:
structure

Structure Summary
Lattice
    abc : 3.994195728785262 3.994195728785262 4.880862
 angles : 90.0 90.0 120.00000000000001
 volume : 67.435075217798
      A : 1.997097864392631 -3.4590749688153366 0.0
      B : 1.997097864392631 3.4590749688153366 0.0
      C : 0.0 0.0 4.880862
    pbc : True True True
PeriodicSite: Al (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Al (1.9971, 1.1530, 3.1632) [0.3333, 0.6667, 0.6481]
PeriodicSite: Al (1.9971, -1.1530, 1.7177) [0.6667, 0.3333, 0.3519]
PeriodicSite: Ni (1.9971, 1.1530, 0.7154) [0.3333, 0.6667, 0.1466]
PeriodicSite: Ni (1.9971, -1.1530, 4.1655) [0.6667, 0.3333, 0.8534]

In [26]:
def get_endmembers_with_templates(template_structure, template_configuration, sublattice_configuration):
    """
    Return endmembers of the structure

    Parameters
    ----------
    template_structure: pymatgen.Structure

    template_configuration: list
        Configuration in the template structure, should be consitent with sublattice model, e.g., one unique
        element for each sublattice. If input manually, make sure be consistent with the sublattice_model_name 

    sublattice_configuration: list
        List of configurations for each sublattice

    Returns
    -------
    endmembers: list 
        List of structures in pymatgen.Structure
    """
    element_dict=dict(map(lambda x, y: [x, y], template_configuration, sublattice_configuration))
    subl_combination=[]
    for comb in product(*map(element_dict.get, sorted(element_dict))):
        subl_combination.append(list(comb))
    endmembers=[]
    for i in subl_combination:
        endmembers.append(substitute_configuration(template_structure, [template_configuration], [i]))
    return subl_combination, endmembers


### 2.2 Analyze sublattice inforamtion

In [27]:
prls=PRLStructure.from_structure(structure)
print('sublattice_site_ratios', prls.sublattice_site_ratios)
print('wyckoff_sites', prls.wyckoff_sites)
#wyckoff_site_list, sublattice_name=get_sublattice_information(structure, use_equivalent_atom=False)
wyckoff_site_list, sublattice_name, sublattice_site_ratio=get_sublattice_information(structure, use_equivalent_atom=True)
template_structure, template_configuration=get_templates(structure, wyckoff_site_list, sublattice_name, equivalent_sites=None)
sublattice_configuration=[['Al','Fe','Ni'],['Al','Fe','Ni'],['Al','Fe','Ni']]
#sublattice_configuration=[['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Fe','Ni'],['Fe','Ni'],['Fe','Ni'],['Fe','Ni'],['Al'],['Al'],['Al'],['Al'],['Al'],['Fe','Ni']]
comb, endmembers=get_endmembers_with_templates(template_structure, template_configuration, sublattice_configuration)

sublattice_site_ratios [1, 2, 2]
wyckoff_sites ['a', 'd', 'd']
Wyckoff sites may not be consistent with equivalent atoms in the structure, please check symmerty information and choose the sublattice model.


### 2.3 Generate and save the target phase endmembers into the directory 

In [28]:
import os

# Create the directory if it doesn't exist
directory_name = str_name+"_endmembers"
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

for i in range(0, len(endmembers)):
    filename = os.path.join(directory_name, str(i+1) +'_'+directory_name +'.POSCAR')
    endmembers[i].sort()
    endmembers[i].to(fmt='poscar', filename=filename)
    firstline = 'POSCAR' + ' ' + 'sublattice information' + ' ' + str(sublattice_name) + str(sublattice_site_ratio) + str(comb[i]) + ' ' + 'generate using OcMAT structure builders\n'
    #print(str(comb[i]))
    with open(filename, 'r', encoding='utf-8') as file:
        data = file.readlines()
        data[0] = firstline
        
    with open(filename, 'w', encoding='utf-8') as file:
        file.writelines(data)


## 3. Generate dilute structures

### 3.1 Analyze sublattice inforamtion

In [29]:
# base on endmember structure list
with open(POSCAR_name, 'r', encoding='utf-8') as file:
    POSCAR_STR = file.read()
structure = Structure.from_str(POSCAR_STR, fmt='POSCAR')
prls=PRLStructure.from_structure(structure)
print('sublattice_site_ratios', prls.sublattice_site_ratios)
print('wyckoff_sites', prls.wyckoff_sites)
wyckoff_site_list, sublattice_name, sublattice_site_ratio=get_sublattice_information(structure, use_equivalent_atom=True)
template_structure, template_configuration=get_templates(structure, wyckoff_site_list, sublattice_name, equivalent_sites=None)
sublattice_configuration=[['Al','Fe','Ni'],['Al','Fe','Ni'],['Al','Fe','Ni']]
#sublattice_configuration=[['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Al'],['Fe','Ni'],['Fe','Ni'],['Fe','Ni'],['Fe','Ni'],['Al'],['Al'],['Al'],['Al'],['Al'],['Fe','Ni']]
comb, endmembers=get_endmembers_with_templates(template_structure, template_configuration, sublattice_configuration)

sublattice_site_ratios [1, 2, 2]
wyckoff_sites ['a', 'd', 'd']
Wyckoff sites may not be consistent with equivalent atoms in the structure, please check symmerty information and choose the sublattice model.


### 3.2 Specifically define the sublattice module of each wyckoff position

In [21]:
#dilute, sublattice_conf=dilute_substitution(endmembers, {'d': ['Al'], 'g': ['Al'],'i': ['Al'],'i2': ['Al'],'i3': ['Al'],'i4': ['Al'],'i5': ['Al'],'i6': ['Al'],'i7': ['Al'],'i8': ['Al'],'i9': ['Fe','Ni'],'i10': ['Fe','Ni'],'i11': ['Fe','Ni'],'i12': ['Fe','Ni'],'j': ['Al'], 'j2': ['Al'],'j3': ['Al'],'j4': ['Al'],'j5': ['Al'],'j6': ['Fe','Ni']})
dilute, sublattice_conf=dilute_substitution(endmembers, {'a': ['Al','Fe','Ni'], 'd': ['Al','Fe','Ni'],'d2': ['Al','Fe','Ni']})

### 3.3 Generate and save the target phase dilute structure into the directory 

In [30]:
import os

# Create the directory if it doesn't exist
directory_name = str_name+"_dilute"
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

for i in range(0, len(dilute)):
    filename = os.path.join(directory_name, str(i+1) +'_'+directory_name +'.POSCAR')
    dilute[i].sort()
    dilute[i].to(fmt='poscar', filename=filename)
    firstline = 'POSCAR' + ' ' + 'sublattice information' +str(sublattice_conf[i]) + 'generate using OcMAT structure builders\n'
    
    with open(filename, 'r', encoding='utf-8') as file:
        data = file.readlines()
        data[0] = firstline
        
    with open(filename, 'w', encoding='utf-8') as file:
        file.writelines(data)

## 4. SQS strucutresgeneration
For ternary BCC SQS structures, you can get the templates from PRL website: https://phaseslab.com/#/resources/.
For Ternary HCP SQS structures, you can generature structures using ATAT SQS database

### 4.1 Get FCC_A1 SQS from SQS ATAT database

In [31]:
from tinydb import TinyDB
dbjson=TinyDB('ATATSQS/ATAT_SQSDB.json')
prototype_name='FCC_A1'
system = 'Al,Fe,Ni'
subl_model=[['Al','Fe','Ni']]
subl_site_ratios=[1]

In [43]:
"""
The sqs module handles converting abstract SQS Structure objects to concrete structures.

The SQS are regular pymatgen Structures with the species named according to sublattice and species type.
These species in pymatgen Structures are named to `Xab`, which corresponds to atom `B` in sublattice `a`.
"""

from __future__ import division

import copy
import itertools

import pymatgen as pmg
from pymatgen.core import Structure


class AbstractSQS(Structure):
    """A pymatgen Structure with special features for SQS.
    """

    def __init__(self, *args, **kwargs):
        """Create a SQS object

        Parameters
        ----------
        args :
            args to pass to Structure
        sublattice_model : [[str]]
            Abstract sublattice model in the ESPEI style, e.g. `[['a', 'b'], ['a']]`.
        sublattice_names : [[str]]
            Names of the sublattices, or the second character in the species names, e.g. `['a', 'c']`.
        kwargs :
            kwargs to pass to Structure
        """
        self.sublattice_model = kwargs.pop('sublattice_model', None)
        self._sublattice_names = kwargs.pop('sublattice_names', None)
        super(AbstractSQS, self).__init__(*args, **kwargs)

    @property
    def normalized_sublattice_site_ratios(self):
        """Return normalized sublattice site ratio. E.g. [[0.25, 0.25], [0.1666, 0.1666, 0.1666]]
        """
        subl_model = self.sublattice_model
        subl_names = self._sublattice_names
        comp_dict = self.composition.as_dict()
        site_ratios = [[comp_dict['X'+name+e+'0+']/self.num_sites for e in subl] for subl, name in zip(subl_model, subl_names)]
        return site_ratios

    @property
    def sublattice_site_ratios(self):
        """Return normalized sublattice site ratio. E.g. [[0.25, 0.25], [0.1666, 0.1666, 0.1666]]
        """
        subl_model = self.sublattice_model
        subl_names = self._sublattice_names
        comp_dict = {k: int(v) for k, v in self.composition.reduced_composition.as_dict().items()}
        site_ratios = [[comp_dict['X'+name+e+'0+'] for e in subl] for subl, name in zip(subl_model, subl_names)]
        return site_ratios

    def get_concrete_sqs(self, subl_model, scale_volume=True):
        """Modify self to be a concrete SQS based on the sublattice model.

        Parameters
        ----------
        subl_model : [[str]]
            List of strings of species names. Must exactly match the shape of self.sublattice_model.
            **Note that order does matter!** [["Al", "Fe"]] and [["Fe", "Al"]] will produce different results!
        scale_volume : bool
            If True, scales the volume of the cell so the ions have at least their minimum atomic radii between them.
        """
        def _subl_error():
            raise ValueError('Concrete sublattice model {} does not match size of abstract sublattice model {}'.format(subl_model, self.sublattice_model))
        if len(subl_model) != len(self.sublattice_model):
            _subl_error()

        # build the replacement dictionary and the site ratios
        # we have to look up the sublattice names to build the replacement species names
        replacement_dict = {}
        site_occupancies = [] # list of [{'FE': 0.3333, 'NI': 0.6666}, {'FE': 1}] for [['FE', 'NI'], ['FE]]
        for abstract_subl, concrete_subl, subl_name, subl_ratios in zip(self.sublattice_model, subl_model, self._sublattice_names, self.sublattice_site_ratios):
            if len(abstract_subl) != len(concrete_subl):
                _subl_error()
            sublattice_ratio_sum = sum(subl_ratios)
            sublattice_occupancy_dict = {}
            for abstract_specie, concrete_specie, site_ratio in zip(abstract_subl, concrete_subl, subl_ratios):
                specie = 'X' + subl_name + abstract_specie
                replacement_dict[specie] = concrete_specie
                sublattice_occupancy_dict[concrete_specie] = sublattice_occupancy_dict.get(concrete_specie, 0) + site_ratio/sublattice_ratio_sum
            site_occupancies.append(sublattice_occupancy_dict)

        # create a copy of myself to make the transformations and make them
        self_copy = copy.deepcopy(self)
        self_copy.replace_species(replacement_dict)

        if scale_volume:
            fractional_comp = dict(self_copy.composition.fractional_composition)
            estimated_density = 0
            for component in self_copy.composition.elements :
                temp = pmg.core.periodic_table.Element(component).data['Density of solid']
                density = float(temp.split(' ')[0])
                estimated_density += (fractional_comp[component] * density)/1000
            self_copy.scale_lattice(float((self_copy.volume/estimated_density)*self_copy.density))

        # finally we will construct the SQS object and set the values for the canonicalized
        # sublattice configuration, site ratios, and site occupancies

        # first, canonicalize the sublattice model, e.g. [['FE', 'FE'], ['NI']] => [['FE'], ['NI']]
        sublattice_configuration = [sorted(set(subl)) for subl in subl_model]
        # construct the sublattice occupancies for the model
        sublattice_occupancies = [[occupancies[specie] for specie in subl] for occupancies, subl in zip(site_occupancies, sublattice_configuration)]
        # sum up the individual sublattice site ratios to the total sublattice ratios.
        # e.g [[0.25, 0.25], [0.1666, 0.1666, 0.1666]] => [0.5, 0.5]
        site_ratios = [sum(ratios) for ratios in self.sublattice_site_ratios]

        # create the SQS and add all of these properties to our SQS
        concrete_sqs = PRLStructure.from_sites(self_copy.sites)
        concrete_sqs.sublattice_configuration = sublattice_configuration
        concrete_sqs.sublattice_occupancies = sublattice_occupancies
        concrete_sqs.sublattice_site_ratios = site_ratios
        return concrete_sqs


    def get_endmember_space_group_info(self, symprec=1e-2, angle_tolerance=5.0):
        """
        Return endmember space group info..

        Args:
            symprec (float): Same definition as in SpacegroupAnalyzer.
                Defaults to 1e-2.
            angle_tolerance (float): Same definition as in SpacegroupAnalyzer.
                Defaults to 5 degrees.

        Returns:
            spacegroup_symbol, international_number
        """
        endmember_subl = [['X' + subl_name for _ in subl] for subl, subl_name in
                          zip(self.sublattice_model, self._sublattice_names)]
        # we need to replace the abstract names with real names of species.
        endmember_speices = {specie for subl in endmember_subl for specie in subl}
        real_species_dict = {abstract_specie: real_specie for abstract_specie, real_specie in
                             zip(endmember_speices, pmg.core.periodic_table._pt_data.keys())}
        # replace them
        endmember_subl = [[real_species_dict[specie] for specie in subl] for subl in endmember_subl]
        # get the structure and spacegroup info
        endmember_struct = self.get_concrete_sqs(endmember_subl, scale_volume=False)
        endmember_space_group_info = endmember_struct.get_space_group_info(symprec=symprec, angle_tolerance=angle_tolerance)
        return endmember_space_group_info

    def as_dict(self, verbosity=1, fmt=None, **kwargs):
        d = super(AbstractSQS, self).as_dict(verbosity=verbosity, fmt=fmt, **kwargs)
        d['sublattice_model'] = self.sublattice_model
        d['sublattice_names'] = self._sublattice_names
        d['sublattice_site_ratios'] = self.sublattice_site_ratios
        endmember_symmetry = self.get_endmember_space_group_info()
        d['symmetry'] = {'symbol': endmember_symmetry[0], 'number': endmember_symmetry[1]}
        return d

    @classmethod
    def from_dict(cls, d, fmt=None):
        sqs = super(AbstractSQS, cls).from_dict(d, fmt=fmt)
        sqs.sublattice_model = d.get('sublattice_model')
        sqs._sublattice_names = d.get('sublattice_names')
        return sqs


def enumerate_sqs(structure, subl_model, scale_volume=True, skip_on_failure=False):
    """
    Return a list of all of the concrete Structure objects from an abstract Structure and concrete sublattice model.
    Parameters
    ----------
    structure : AbstractSQS
        SQS object. Must be abstract.
    subl_model : [[str]]
        List of strings of species names, in the style of ESPEI `input.json`. This sublattice model
        can be of higher dimension than the SQS, e.g. a [["Al", "Fe", "Ni"]] for a fcc 75/25 binary SQS
        will generate the following Structures:
        Al0.75Fe0.25, Al0.75Ni0.25      Fe0.75Al0.25, Fe0.75Ni0.25      Ni0.75Al0.25, Ni0.75Fe0.25
        *Note that the ordering of species the sublattice model does not matter!*
    scale_volume : bool
        If True, scales the volume of the cell so the ions have at least their minimum atomic radii between them.
    skip_on_failure : bool
        If True, will skip if the sublattice model is lower order and return [] instead of raising

    Returns
    -------
    [PRLStructure]
        List of all concrete PRLStructure objects that can be created from the sublattice model.
    """
    if len(subl_model) != len(structure.sublattice_model):
        raise ValueError('Passed sublattice model ({}) does not agree with the passed structure ({})'.format(subl_model, structure.sublattice_model))
    possible_subls = []
    for subl, abstract_subl in zip(subl_model, structure.sublattice_model):
        subls = itertools.product(subl, repeat=len(abstract_subl))
        possible_subls.append(subls)
    unique_subl_models = itertools.product(*possible_subls)

    # create a list of unique concrete structures with the generated sublattice models
    unique_sqs = []
    unique_configurations_occupancies = []
    for model in unique_subl_models:
        proposed_sqs = structure.get_concrete_sqs(model, scale_volume)
        proposed_config_occupancy = (proposed_sqs.sublattice_configuration, proposed_sqs.sublattice_occupancies)
        if proposed_config_occupancy not in unique_configurations_occupancies:
            unique_configurations_occupancies.append(proposed_config_occupancy)
            unique_sqs.append(proposed_sqs)
    return unique_sqs


### 4.2 Analyze sublattice inforamtion

In [44]:

absqs=get_structures_from_database(dbjson, prototype_name, subl_model, subl_site_ratios)
concrete_structure={}
absqs_pymatgen={}
for j in range(0,len(absqs)):
    absqs_pymatgen[j]=AbstractSQS.from_dict(absqs[j])
    concrete_structure[j] = enumerate_sqs(absqs_pymatgen[j], subl_model)
print(absqs_pymatgen)

{0: Structure Summary
Lattice
    abc : 1.224744871391589 2.1213203435596424 3.4641016151377544
 angles : 90.0 90.0 90.0
 volume : 9.0
      A : 0.5 -0.5 -1.0
      B : -1.5 -1.5 0.0
      C : -2.0 2.0 -2.0
    pbc : True True True
PeriodicSite: Xaa0+ (-1.5000, -1.0000, -0.5000) [0.1667, 0.8333, 0.1667]
PeriodicSite: Xaa0+ (-3.0000, 0.5000, -2.5000) [0.5000, 0.8333, 1.0000]
PeriodicSite: Xaa0+ (-2.5000, 0.5000, -3.0000) [1.0000, 0.6667, 1.0000]
PeriodicSite: Xac0+ (-2.5000, 0.0000, -1.5000) [0.1667, 0.8333, 0.6667]
PeriodicSite: Xac0+ (-2.0000, -0.5000, -1.5000) [0.5000, 0.8333, 0.5000]
PeriodicSite: Xaa0+ (-1.5000, -0.5000, -2.0000) [1.0000, 0.6667, 0.5000]
PeriodicSite: Xab0+ (-1.5000, -1.0000, -1.5000) [0.8333, 0.8333, 0.3333]
PeriodicSite: Xab0+ (-1.5000, -0.5000, -1.0000) [0.3333, 0.6667, 0.3333]
PeriodicSite: Xaa0+ (-1.0000, -1.0000, -1.0000) [0.6667, 0.6667, 0.1667]
PeriodicSite: Xab0+ (-1.0000, -0.5000, -0.5000) [0.1667, 0.5000, 0.1667]
PeriodicSite: Xab0+ (-2.5000, 1.0000, -2.

### 4.3 Check the list of the SQS structures

In [101]:
a=6
for i in range(0, len(concrete_structure[a])):
    #print(i)
    #print(type(concrete_structure))
    print(concrete_structure[a][0]) #tenary hcp sqs: 1, 4 ; 1,7; 1,11; 5 4;

Full Formula (Al32)
Reduced Formula: Al
abc   :   6.401877   9.495519   9.917746
angles:  64.202352  82.582444  78.330854
pbc   :       True       True       True
Sites (32)
  #  SP         a       b        c
---  ----  ------  ------  -------
  0  Al    1       1       1
  1  Al    0.5625  0.0625  0.09375
  2  Al    0.125   0.125   0.1875
  3  Al    1       1       0.5
  4  Al    0.6875  0.1875  0.28125
  5  Al    0.375   0.375   0.0625
  6  Al    0.5625  0.0625  0.59375
  7  Al    0.25    0.25    0.375
  8  Al    0.125   0.125   0.6875
  9  Al    0.9375  0.4375  0.15625
 10  Al    0.8125  0.3125  0.46875
 11  Al    0.5     0.5     0.25
 12  Al    0.1875  0.6875  0.03125
 13  Al    0.6875  0.1875  0.78125
 14  Al    0.375   0.375   0.5625
 15  Al    0.0625  0.5625  0.34375
 16  Al    0.25    0.25    0.875
 17  Al    0.75    0.75    0.125
 18  Al    0.9375  0.4375  0.65625
 19  Al    0.625   0.625   0.4375
 20  Al    0.3125  0.8125  0.21875
 21  Al    0.8125  0.3125  0.96875
 22  Al   

### 3.3 Generate and save the target phase SQS structures into the directory 

In [45]:
import os
from pymatgen.core import Structure

# List of cases to go through
cases = [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0,5), (0,6), (0,7),(0,8),(0,9),
         (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1,5), (1,6), (1,7),(1,8),
         (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2,5), (2,6), (2,7),(2,8),
         (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3,5), (3,6), (3,7),(3,8),
         (4, 0), (4, 1), (4, 2),
         (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5,5),
         (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6,5),(6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6,11),(6, 12), (6, 13), (6,14)]

# Directory to save POSCAR files
directory_name = "FCC_SQS"
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

# Iterate over each case
for case in cases:
    # Extract structure from concrete_structure
    structure_dict = concrete_structure[case[0]][case[1]].as_dict()
    structure = Structure.from_dict(structure_dict)

    # Convert to POSCAR format string and then create sorted Structure object
    poscar_str = structure.to(fmt='poscar')
    poscar_sorted = Structure.from_str(poscar_str, fmt='POSCAR', sort=True)

    # Create filename for the POSCAR file
    filename = os.path.join(directory_name, f"FCCSQS_{case[0]}_{case[1]}.POSCAR")

    # Save the sorted structure to a file in POSCAR format
    poscar_sorted.to(filename=filename, fmt='poscar')

    # Optionally, print the structure or filename
    print(f"Saved POSCAR file for case {case} as {filename}")


Saved POSCAR file for case (0, 0) as FCC_SQS/FCCSQS_0_0.POSCAR
Saved POSCAR file for case (0, 1) as FCC_SQS/FCCSQS_0_1.POSCAR
Saved POSCAR file for case (0, 2) as FCC_SQS/FCCSQS_0_2.POSCAR
Saved POSCAR file for case (0, 3) as FCC_SQS/FCCSQS_0_3.POSCAR
Saved POSCAR file for case (0, 4) as FCC_SQS/FCCSQS_0_4.POSCAR
Saved POSCAR file for case (0, 5) as FCC_SQS/FCCSQS_0_5.POSCAR
Saved POSCAR file for case (0, 6) as FCC_SQS/FCCSQS_0_6.POSCAR
Saved POSCAR file for case (0, 7) as FCC_SQS/FCCSQS_0_7.POSCAR
Saved POSCAR file for case (0, 8) as FCC_SQS/FCCSQS_0_8.POSCAR
Saved POSCAR file for case (0, 9) as FCC_SQS/FCCSQS_0_9.POSCAR
Saved POSCAR file for case (1, 0) as FCC_SQS/FCCSQS_1_0.POSCAR
Saved POSCAR file for case (1, 1) as FCC_SQS/FCCSQS_1_1.POSCAR
Saved POSCAR file for case (1, 2) as FCC_SQS/FCCSQS_1_2.POSCAR
Saved POSCAR file for case (1, 3) as FCC_SQS/FCCSQS_1_3.POSCAR
Saved POSCAR file for case (1, 4) as FCC_SQS/FCCSQS_1_4.POSCAR
Saved POSCAR file for case (1, 5) as FCC_SQS/FCCSQS_1_5

## 5. Execrsie: try it by your self: generate the SQS BCC from ATAT database

### Following is the solution of gernerating BCC SQS structures

In [106]:
from tinydb import TinyDB
dbjson=TinyDB('ATATSQS/ATAT_SQSDB.json')
prototype_name='BCC_A2'
system = 'Al,Fe,Ni'
subl_model=[['Al','Fe','Ni']]
subl_site_ratios=[1]


In [42]:
absqs=get_structures_from_database(dbjson, prototype_name, subl_model, subl_site_ratios)
concrete_structure={}
absqs_pymatgen={}
for j in range(0,len(absqs)):
    absqs_pymatgen[j]=AbstractSQS.from_dict(absqs[j])
    concrete_structure[j] = enumerate_sqs(absqs_pymatgen[j], subl_model)
print(absqs_pymatgen)

{0: Structure Summary
Lattice
    abc : 1.224744871391589 2.1213203435596424 3.4641016151377544
 angles : 90.0 90.0 90.0
 volume : 9.0
      A : 0.5 -0.5 -1.0
      B : -1.5 -1.5 0.0
      C : -2.0 2.0 -2.0
    pbc : True True True
PeriodicSite: Xaa0+ (-1.5000, -1.0000, -0.5000) [0.1667, 0.8333, 0.1667]
PeriodicSite: Xaa0+ (-3.0000, 0.5000, -2.5000) [0.5000, 0.8333, 1.0000]
PeriodicSite: Xaa0+ (-2.5000, 0.5000, -3.0000) [1.0000, 0.6667, 1.0000]
PeriodicSite: Xac0+ (-2.5000, 0.0000, -1.5000) [0.1667, 0.8333, 0.6667]
PeriodicSite: Xac0+ (-2.0000, -0.5000, -1.5000) [0.5000, 0.8333, 0.5000]
PeriodicSite: Xaa0+ (-1.5000, -0.5000, -2.0000) [1.0000, 0.6667, 0.5000]
PeriodicSite: Xab0+ (-1.5000, -1.0000, -1.5000) [0.8333, 0.8333, 0.3333]
PeriodicSite: Xab0+ (-1.5000, -0.5000, -1.0000) [0.3333, 0.6667, 0.3333]
PeriodicSite: Xaa0+ (-1.0000, -1.0000, -1.0000) [0.6667, 0.6667, 0.1667]
PeriodicSite: Xab0+ (-1.0000, -0.5000, -0.5000) [0.1667, 0.5000, 0.1667]
PeriodicSite: Xab0+ (-2.5000, 1.0000, -2.

In [109]:
a=6
for i in range(0, len(concrete_structure[a])):
    #print(i)
    #print(type(concrete_structure))
    print(concrete_structure[a][14]) #0 4;6 4 ;6 7; 6 11

Full Formula (Ni32)
Reduced Formula: Ni
abc   :   7.911176   7.911176   6.851279
angles: 106.778655 106.778655  60.000000
pbc   :       True       True       True
Sites (32)
  #  SP         a       b      c
---  ----  ------  ------  -----
  0  Ni    0.125   0.125   0.75
  1  Ni    0.3125  0.3125  0.875
  2  Ni    0.125   0.625   0.75
  3  Ni    0.3125  0.8125  0.875
  4  Ni    0.0625  0.0625  0.375
  5  Ni    0.25    0.25    0.5
  6  Ni    0.0625  0.5625  0.375
  7  Ni    0.625   0.125   0.75
  8  Ni    0.4375  0.4375  0.625
  9  Ni    0.25    0.75    0.5
 10  Ni    0.8125  0.3125  0.875
 11  Ni    0.625   0.625   0.75
 12  Ni    0.4375  0.9375  0.625
 13  Ni    0.8125  0.8125  0.875
 14  Ni    1       1       1
 15  Ni    0.1875  0.1875  0.125
 16  Ni    1       0.5     1
 17  Ni    0.5625  0.0625  0.375
 18  Ni    0.375   0.375   0.25
 19  Ni    0.1875  0.6875  0.125
 20  Ni    0.75    0.25    0.5
 21  Ni    0.5625  0.5625  0.375
 22  Ni    0.375   0.875   0.25
 23  Ni    0.9375  0.

In [111]:
import os
from pymatgen.core import Structure

# List of cases to go through
cases = [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0,5), (0,6), (0,7),(0,8),(0,9),
         (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1,5), (1,6), (1,7),(1,8),
         (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2,5), (2,6), (2,7),(2,8),
         (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3,5), (3,6), (3,7),(3,8),
         (4, 0), (4, 1), (4, 2),
         (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5,5),
         (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6,5),(6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6,11),(6, 12), (6, 13), (6,14)]

# Directory to save POSCAR files
directory_name = "BCC_SQS"
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

# Iterate over each case
for case in cases:
    # Extract structure from concrete_structure
    structure_dict = concrete_structure[case[0]][case[1]].as_dict()
    structure = Structure.from_dict(structure_dict)

    # Convert to POSCAR format string and then create sorted Structure object
    poscar_str = structure.to(fmt='poscar')
    poscar_sorted = Structure.from_str(poscar_str, fmt='POSCAR', sort=True)

    # Create filename for the POSCAR file
    filename = os.path.join(directory_name, f"BCCSQS_{case[0]}_{case[1]}.POSCAR")

    # Save the sorted structure to a file in POSCAR format
    poscar_sorted.to(filename=filename, fmt='poscar')

    # Optionally, print the structure or filename
    print(f"Saved POSCAR file for case {case} as {filename}")

Saved POSCAR file for case (0, 0) as BCC_SQS/BCCSQS_0_0.POSCAR
Saved POSCAR file for case (0, 1) as BCC_SQS/BCCSQS_0_1.POSCAR
Saved POSCAR file for case (0, 2) as BCC_SQS/BCCSQS_0_2.POSCAR
Saved POSCAR file for case (0, 3) as BCC_SQS/BCCSQS_0_3.POSCAR
Saved POSCAR file for case (0, 4) as BCC_SQS/BCCSQS_0_4.POSCAR
Saved POSCAR file for case (0, 5) as BCC_SQS/BCCSQS_0_5.POSCAR
Saved POSCAR file for case (0, 6) as BCC_SQS/BCCSQS_0_6.POSCAR
Saved POSCAR file for case (0, 7) as BCC_SQS/BCCSQS_0_7.POSCAR
Saved POSCAR file for case (0, 8) as BCC_SQS/BCCSQS_0_8.POSCAR
Saved POSCAR file for case (0, 9) as BCC_SQS/BCCSQS_0_9.POSCAR
Saved POSCAR file for case (1, 0) as BCC_SQS/BCCSQS_1_0.POSCAR
Saved POSCAR file for case (1, 1) as BCC_SQS/BCCSQS_1_1.POSCAR
Saved POSCAR file for case (1, 2) as BCC_SQS/BCCSQS_1_2.POSCAR
Saved POSCAR file for case (1, 3) as BCC_SQS/BCCSQS_1_3.POSCAR
Saved POSCAR file for case (1, 4) as BCC_SQS/BCCSQS_1_4.POSCAR
Saved POSCAR file for case (1, 5) as BCC_SQS/BCCSQS_1_5